In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

#
# data
#
from techminer import *


# Records(directory="../data/processed/debug").import_records(source="../data/tmp/debug-scopus.csv", filetype="scopus")
# Records(directory="../data/processed/debug").process_records()
# Records(directory="../data/processed/debug").coverage()
# Records(directory="../data/processed/debug").describe()
# Records(directory="../data/processed/debug").core_sources()
# Records(directory="../data/processed/debug").core_authors()

# wos_df = load_file(filepath="techminer/tests/data/savedrecs.txt", filetype="wos", datastorepath="./datastore")
# dim_df = load_file(filepath="techminer/tests/data/dimensions.csv", filetype="dimensions", datastorepath="./datastore")

In [1]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import *

# core_sources(directory_or_records="../data/processed/debug")
# core_authors(directory_or_records="../data/processed/debug")
# coverage(directory_or_records="../data/processed/debug")
# most_cited_documents(directory_or_records="../data/processed/debug")
# summary(directory_or_records="../data/processed/debug")
# worldmap(count_documents_by_term(directory_or_records="../data/processed/debug", column="countries"))
bradford_plot(directory_or_records="../data/processed/debug")

,authors,pub_year,document_title,publication_name,record_id,global_citations
0,Abraham MJ; Murtola T; Schulz R; Páll S; Smith...,2015,Gromacs: High performance molecular simulation...,SOFTWAREX,2015-0001,4746
1,Cooper SJ; Bertei A; Shearing PR; Kilner JA; B...,2016,TauFactor: An open-source application for calc...,SOFTWAREX,2016-0001,106
2,Lehtola S; Steigemann C; Oliveira MJT; Marques...,2018,Recent developments in LIBXC — A comprehensive...,SOFTWAREX,2018-0001,102
3,Loizides C; Nagle J; Steinberg P,2015,Improved version of the PHOBOS Glauber Monte C...,SOFTWAREX,2015-0002,99
4,Sundararaman R; Letchworth-Weaver K; Schwarz K...,2017,JDFTx: Software for joint density-functional t...,SOFTWAREX,2017-0001,67
5,Tudisco E; Andò E; Cailletaud R; Hall SA,2017,TomoWarp2: A local digital volume correlation ...,SOFTWAREX,2017-0002,41
6,Govender N; Wilke DN; Kok S,2015,Blaze-DEMGPU: Modular high performance DEM fra...,SOFTWAREX,2015-0003,32
7,Vanfretti L; Rabuzin T; Baudette M; Murad M,2016,iTesla Power Systems Library (iPSL): A Modelic...,SOFTWAREX,2016-0002,32
8,Binder JM; Stark A; Tomek N; Scheuer J; Frank ...,2017,Qudi: A modular python suite for experiment co...,SOFTWAREX,2017-0003,32
9,Oishi AC; Hawthorne DA; Oren R,2016,"Baseliner: An open-source, interactive tool fo...",SOFTWAREX,2016-0003,30


In [ ]:
count_documents_by_term(directory_or_records="../data/processed/debug", column="country")

In [ ]:
#
# query module
#
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.query import * 


# count_documents_by_term(directory_or_records="../data/processed/debug", column='authors', sep='; ')
# count_documents_by_year(directory_or_records="../data/processed/debug")
# count_global_citations_by_term(directory_or_records="../data/processed/debug", column='authors', sep='; ')
# count_global_citations_by_year(directory_or_records="../data/processed/debug")
# count_local_citations_by_term(directory_or_records="../data/processed/debug", column='authors', sep='; ')
# count_local_citations_by_year(directory_or_records="../data/processed/debug")
# count_terms_by_column(directory_or_records="../data/processed/debug", sep="; ", column='authors')
# mean_global_citations_by_year(directory_or_records="../data/processed/debug")
# mean_local_citations_by_year(directory_or_records="../data/processed/debug")
# term_analysis(directory_or_records="../data/processed/debug", column='authors')
# time_analysis(directory_or_records="../data/processed/debug")

# termino por año <----

# most_cited_documents(directory_or_records="../data/processed/debug")
# compute_impact_index(directory_or_records="../data/processed/debug", column='authors')




# ---
# worldmap(count_documents_by_term(directory_or_records="../data/processed/debug", column='countries'))


In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.data import *

# coverage(directory_or_records="../data/processed/debug")
describe_records(directory_or_records="../data/processed/debug")

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

#
# features
#
from techminer.features import *

# extract_coutries(directory_or_records="../data/processed/debug", column='affiliations')






In [ ]:
#
# query modules
#
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.query import * 
from techminer.visualization import worldmap



# term_analysis(directory_or_records="../data/processed/debug", column='authors')
### count_terms_by_column(directory_or_records="../data/processed/debug", sep="; ", column='authors')
# time_analysis(directory_or_records="../data/processed/debug")
### count_local_citations_by_year(directory_or_records="../data/processed/debug")
### count_global_citations_by_year(directory_or_records="../data/processed/debug")
### count_documents_by_year(directory_or_records="../data/processed/debug")
### mean_global_citations_by_year(directory_or_records="../data/processed/debug")
# most_cited_documents(directory_or_records="../data/processed/debug")
# compute_impact_index(directory_or_records="../data/processed/debug", column='authors')




# ---
# worldmap(count_documents_by_term(directory_or_records="../data/processed/debug", column='countries'))



In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer import Records

Records(directory)

# core_sources(directory_or_records="../data/processed/debug")
# core_authors(directory_or_records="../data/processed/debug")
# lotka(core_authors(directory_or_records="../data/processed/debug"), colormap='Blues', figsize=(6,3))


In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)


from techminer.visualization import bradford

bradford(directory_or_records="../data/processed/debug")

In [ ]:
import pandas as pd

from techminer.data.records import load_records

# sorted(load_records(directory="../data/processed/debug").columns)
load_records(directory="../data/processed/debug")

In [ ]:
from src.features.apply_keywords_thesaurus import apply_keywords_thesaurus
from src.features.apply_institutions_thesaurus import apply_institutions_thesaurus

# Ok!
apply_keywords_thesaurus(datastoredir="../data/processed/debug")
apply_institutions_thesaurus(datastoredir="../data/processed/debug")

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from techminer.analysis import Bradford

bradford = Bradford(datastoredir="../data/processed/debug")

bradford.plot(max_items=8, cmap='Blues', figsize=(8, 5), fontsize=9)

In [ ]:
bradford.sources_

In [ ]:
import os
import sys

currentdir = os.getcwd()
parentdir = os.path.dirname(currentdir)
sys.path.append(parentdir)

from src.analysis import TopDocuments

TopDocuments(datastoredir="../data/processed/debug").detailed_

In [ ]:
from techminer.analysis.top_documents import TopDocuments

TopDocuments(datastorepath="./datastore").print()

In [ ]:
from techminer.analysis.core_authors import CoreAuthors

CoreAuthors(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.core_authors import CoreAuthors

CoreAuthors(datastorepath="./datastore").plot(figsize=(8, 4))

In [ ]:
from techminer.analysis.core_sources import CoreSources

CoreSources(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.coverage import Coverage

Coverage(datastorepath="./datastore").table_

In [ ]:
from techminer.analysis.worldmap import WorldMap

WorldMap(datastorepath="./datastore").plot(figsize=(9, 5), colormap='Blues')

In [ ]:
import pandas as pd

pd.read_csv("../data/processed/debug/records.csv").columns